# 프로젝트: 멋진 작사가 만들기

## 1. 데이터 다운로드

In [1]:
# ln -s ~/data

## 2. 데이터 읽어오기

In [2]:
import glob
import os

txt_file_path = os.getenv('HOME') + '/aiffel/exploration/E04/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10]) # \n : 줄바꿈

데이터 크기: 187088
Examples:
 ['I hear you callin\', "Here I come baby"', 'To save you, oh oh', "Baby no more stallin'", 'These hands have been longing to touch you baby', "And now that you've come around, to seein' it my way", "You won't regret it baby, and you surely won't forget it baby", "It's unbelieveable how your body's calling for me", "I can just hear it callin' callin' for me My body's callin' for you", "My body's callin' for you", "My body's callin' for you"]


## 3. 데이터 정제하기

### 공백 문장 여부 체크하기

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
    
    if idx > 9: break
    
    print(sentence)

I hear you callin', "Here I come baby"
To save you, oh oh
Baby no more stallin'
These hands have been longing to touch you baby
And now that you've come around, to seein' it my way
You won't regret it baby, and you surely won't forget it baby
It's unbelieveable how your body's calling for me
I can just hear it callin' callin' for me My body's callin' for you
My body's callin' for you
My body's callin' for you


### 토큰화(Tokenize)

#### 정규 표현식을 이용한 corpus 생성

In [4]:
import re

# 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
# 2. 특수문자 양쪽에 공백을 넣고
# 3. 여러개의 공백은 하나의 공백으로 바꿉니다
# 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
# 5. 다시 양쪽 공백을 지웁니다
# 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다

# 데이터 정제를 위한 정규 표현식 적용 함수 만들기
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()  # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)  # 2
    sentence = re.sub(r'[" "]+', " ", sentence)  # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # 4
    sentence = re.sub(r'\[[^)]*\]', '', sentence)
    sentence = sentence.strip()  # 5
    sentence = '<start> ' + sentence + ' <end>'  # 6
    return sentence

# 예시 문장 필터링 확인
print(preprocess_sentence(" Don't  @_won't ;;;sample        sentence."))

<start> don t won t sample sentence . <end>


#### 소스 문장, 타겟 문장 만들기

In [5]:
# 정제된 문장 모으기
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장은 건너 뛰기
    if len(sentence) == 0: continue
    if len(sentence.split()) > 13: continue  # 토큰의 개수가 13개가 넘어가면 학습 데이터에서 제외
        
    # 정제된 문장 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
# 정제된 문장 샘플 확인(10개)
corpus[:10]

['<start> i hear you callin , here i come baby <end>',
 '<start> to save you , oh oh <end>',
 '<start> baby no more stallin <end>',
 '<start> these hands have been longing to touch you baby <end>',
 '<start> and now that you ve come around , to seein it my way <end>',
 '<start> you won t regret it baby , and you surely won t forget it baby <end>',
 '<start> it s unbelieveable how your body s calling for me <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you tell me , what s your desire <end>']

#### 텐서플로우 패키지를 이용해 벡터화

In [6]:
import tensorflow as tf
# 토큰화 : Tokenizer, pad_sequences 사용

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000,    # 12,000개의 단어장 tokenizer 만들기
        filters = ' ',        # 문장을 이미 정제했기 때문에 filters 미사용
        oov_token = "<unk>")  # 12,000개 단어에 포함되지 못하는 단어는 <unk>로 대체
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰주는 작업 (maxlen : 15)
    # 만약 시퀀스가 짧다면 문장 뒤에 padding을 붙여 길이를 맞춰줌
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   5 187 ...   0   0   0]
 [  2  10 576 ...   0   0   0]
 [  2  51  38 ...   0   0   0]
 ...
 [  2   5  93 ...   0   0   0]
 [  2   9 157 ...   0   0   0]
 [  2 164  15 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f6805346b50>


In [7]:
# 생성된 tensor 데이터를 3번째 행, 10번째 열까지만 출력해본다.
print(tensor[:3, :10])

# tensor 크기 확인
tensor.shape

[[   2    5  187    7  821    4   92    5   67   51]
 [   2   10  576    7    4   41   41    3    0    0]
 [   2   51   38   99 7077    3    0    0    0    0]]


(163272, 15)

텐서 데이터는 모두 정수로 이루어져 있는데, tokenizer에 구축된 단어 사전의 인덱스 단어 사전이 어떻게 구축되었는지 아래와 같이 확인한다.

In [8]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 15: break

# 사전에는 없지만 0은 바로 패딩 문자 <pad>가 될 것

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t


In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[  2   5 187   7 821   4  92   5  67  51   3   0   0   0]
[  5 187   7 821   4  92   5  67  51   3   0   0   0   0]


In [10]:
src_input.shape

(163272, 14)

In [11]:
tgt_input.shape

(163272, 14)

## 4. 평가 데이터셋 분리

In [12]:
from sklearn.model_selection import train_test_split

# tensor를 train, test 데이터셋으로 각각 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2)

print('enc_train 개수: ', len(enc_train))
print('dec_train 개수: ', len(dec_train))

enc_train 개수:  130617
dec_train 개수:  130617


In [13]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (130617, 14)
Target Train: (130617, 14)


In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
step_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 5. 인공지능 만들기

In [15]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [16]:
# 데이터셋에서 데이터 한 배치만 불러온다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어본다.
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 7.81740164e-05, -2.82949732e-05, -1.44942474e-04, ...,
          2.94208992e-04, -2.20224552e-04, -1.20946424e-04],
        [ 1.63521938e-04, -2.28375884e-05,  2.68275180e-05, ...,
          4.17186850e-04, -4.19628457e-04, -1.41565579e-05],
        [ 2.99146544e-04,  1.62833036e-04, -1.74058310e-04, ...,
          6.37780468e-04, -1.96775931e-04, -2.73265669e-05],
        ...,
        [-2.25254800e-04,  6.48849586e-04, -3.01928783e-04, ...,
          8.35007115e-04, -4.90190578e-04,  6.37940946e-04],
        [-2.95354635e-04,  4.40775329e-04, -1.48666499e-04, ...,
          5.19652502e-04, -1.23682595e-03,  8.96741985e-04],
        [-3.38146579e-04,  2.00698618e-04,  2.79358410e-05, ...,
          1.86800011e-04, -1.94472261e-03,  1.15084101e-03]],

       [[ 7.81740164e-05, -2.82949732e-05, -1.44942474e-04, ...,
          2.94208992e-04, -2.20224552e-04, -1.20946424e-04],
        [ 9.84691942e-05,  2.97257880e-04, -2

In [17]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [18]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
637/637 [==============================] - 223s 347ms/step - loss: 3.9827
Epoch 2/10
637/637 [==============================] - 225s 353ms/step - loss: 3.0767
Epoch 3/10
637/637 [==============================] - 226s 354ms/step - loss: 2.8721
Epoch 4/10
637/637 [==============================] - 226s 354ms/step - loss: 2.7264
Epoch 5/10
637/637 [==============================] - 225s 354ms/step - loss: 2.6024
Epoch 6/10
637/637 [==============================] - 227s 356ms/step - loss: 2.4908
Epoch 7/10
637/637 [==============================] - 227s 355ms/step - loss: 2.3827
Epoch 8/10
637/637 [==============================] - 226s 355ms/step - loss: 2.2862
Epoch 9/10
637/637 [==============================] - 226s 355ms/step - loss: 2.1920
Epoch 10/10
637/637 [==============================] - 226s 354ms/step - loss: 2.1183


## 6. 학습 결과 평가하기

In [19]:
def generate_text(model, tokenizer, init_sentence="<strat>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    
    while True:
        # 1
        predict = model(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    
    return generated

In [21]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

## 회고

### 새로 알게된 것
- 자연어 처리를 위해서는 RNN, LSTM, 임베딩 등을 이해하여야 한다.
- 복잡하고 헷갈리는 정규표현식을 '이렇게 사용할 수 있구나'라고 알게 되었다. 엑셀에서는 ctrl+h 만 누르면 단어, 숫자, 특수문자 등을 아주 쉽게 바꿀 수 있는데 파이썬은 이 부분이 조금 어려운 것 같다. 그렇지만 데이터 분석시에도 많이 활용되는 부분이어서 반드시 알아두어야 할 부분이다.

### 아쉬운 점
- 지난 주에 진행 했었던 E03, E04 노드는 풀잎스쿨 발표 준비와 겹쳐서 루브릭에 맞추기에만 급급했다. 운좋게도 한번 만에 Validation loss가 2.2 이하로 나왔으나 코드를 이해 못한 부분이 많다.
- [Hook]과 같은 대괄호를 제거하기 위해 전처리 과정에서 정규표현식을 추가적으로 사용해서 제거 하였는데 생각보다 제거되는 것이 많지 않아 학습데이터 개수가 124960개 보다 많이 나왔다.

### 소감
- 다른 파일에서 학습 후 다양한 단어를 입력하여 작사를 하게 했는데 꽤 그럴듯 했다.

### 참고자료
- [딥러닝을 이용한 자연어 처리 입문]('https://wikidocs.net/book/2155')
- [RNN과 LSTM을 이해해보자!]('https://ratsgo.github.io/natural%20language%20processing/2017/03/09/rnnlstm/')